In [1]:
import torch
model = torch.hub.load('facebookresearch/pytorchvideo','slowfast_r101', pretrained=True)

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from torchvision.transforms import Compose, Lambda
import urllib.request
import json
from typing import Dict

Using cache found in C:\Users\bruse/.cache\torch\hub\facebookresearch_pytorchvideo_main
h:\RUTUBE_HACK\venv\Lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
h:\RUTUBE_HACK\venv\Lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [3]:
# Установка устройства: GPU или CPU
device = "cpu"
model = model.eval()
model = model.to(device)

# Загрузка названий классов Kinetics
json_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
json_filename = "kinetics_classnames.json"
try:
    urllib.request.urlretrieve(json_url, json_filename)
except Exception as e:
    print(f"Ошибка при загрузке файла классов: {e}")

with open(json_filename, "r") as f:
    kinetics_classnames = json.load(f)

# Создание отображения id в названия классов
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[int(v)] = k

# Параметры преобразований
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4

categories = {
    # Транспорт
    'Транспорт': ['changing wheel', 'driving car', 'driving tractor', 'riding scooter', 'golf driving', 'riding a bike', 'pushing car', 'motorcycling', 'unloading truck', 'changing oil', 'checking tires'],
    # Книги и литература
    'Книги и литература': ['sharpening pencil', 'reading newspaper', 'reading book'],
    # Бизнес и финансы
    'Бизнес и финансы': ['counting money', 'auctioning'],
    # Карьера
    'Карьера': [],
    # Образование
    'Образование': ['writing'],
    # События и достопримечательности
    'События и достопримечательности': ['clapping', 'drinking beer', 'tasting beer', 'balloon blowing', 'bartending', 'bowling', 'playing poker', 'celebrating', 'drinking shots', 'applauding', 'smoking hookah'],
    # Семья и отношения
    'Семья и отношения': ['flying kite', 'blowing out candles', 'braiding hair', 'hopscotch', 'ironing', 'dining', 'making snowman', 'kissing', 'crawling baby', 'carrying baby', 'baby waking up'],
    # Изобразительное искусство
    'Изобразительное искусство': ['somersaulting', 'sharpening pencil', 'spray painting', 'brush painting'],
    # Еда и напитки
    'Еда и напитки': ['sharpening knives', 'eating ice cream', 'eating carrots', 'cooking on campfire', 'breading or breadcrumbing', 'drinking', 'cooking egg', 'eating spaghetti', 'making pizza', 'peeling potatoes', 'flipping pancake', 'drinking beer', 'tasting beer', 'making sushi', 'making a sandwich', 'cutting pineapple', 'bartending', 'frying vegetables', 'eating watermelon', 'eating doughnuts', 'drinking shots', 'peeling apples', 'eating burger', 'eating hotdog', 'scrambling eggs', 'cooking chicken', 'cooking sausages', 'tossing salad', 'barbequing', 'cutting watermelon', 'making tea', 'tasting food', 'making a cake', 'eating chips', 'baking cookies', 'grinding meat', 'eating cake'],
    # Здоровый образ жизни
    'Здоровый образ жизни': ['bench pressing', 'deadlifting', 'eating carrots', 'playing badminton', 'jogging', 'running on treadmill', 'push up', 'pull ups', 'skipping rope'],
    # Хобби и интересы
    'Хобби и интересы': ['sharpening knives', 'cutting nails', 'playing flute', 'playing saxophone', 'abseiling', 'rock climbing', 'golf putting', 'tai chi', 'skydiving', 'ice fishing', 'playing chess', 'water sliding', 'roller skating', 'snowboarding', 'catching fish', 'playing paintball'],
    # Дом и сад
    'Дом и сад': ['bee keeping', 'peeling potatoes', 'driving tractor', 'feeding goats', 'sanding floor', 'trimming trees', 'mowing lawn', 'decorating the christmas tree', 'shoveling snow', 'planting trees', 'watering plants', 'digging', 'arranging flowers', 'chopping wood'],
    # Медицина
    'Медицина': ['sticking tongue out', 'bandaging', 'pushing wheelchair', 'gargling', 'massaging person\'s head', 'brushing teeth'],
    # Фильмы и анимация
    'Фильмы и анимация': ['smoking', 'kissing', 'crying', 'laughing'],
    # Музыка и аудио
    'Музыка и аудио': ['playing flute', 'playing saxophone', 'playing cymbals', 'playing organ', 'playing bagpipes', 'playing clarinet', 'playing piano', 'dancing charleston', 'playing accordion', 'playing trumpet', 'recording music', 'drumming fingers', 'playing ukulele', 'playing violin', 'playing drums', 'playing harmonica', 'playing xylophone', 'playing cello', 'singing', 'tapping guitar', 'strumming guitar', 'playing guitar', 'playing harp', 'beatboxing'],
    # Новости и политика
    'Новости и политика': ['extinguishing fire', 'presenting weather forecast', 'news anchoring', 'reading newspaper'],
    # Личные финансы
    'Личные финансы': [],
    # Животные
    'Животные': ['milking cow', 'holding snake', 'bee keeping', 'feeding goats', 'petting cat', 'training dog', 'grooming dog', 'riding elephant', 'grooming horse', 'petting animal (not cat)', 'riding mule', 'walking the dog', 'shearing sheep', 'feeding birds'],
    # Недвижимость
    'Недвижимость': [],
    # Религия и духовность
    'Религия и духовность': [],
    # Наука
    'Наука': [],
    # Покупки
    'Покупки': [],
    # Спорт
    'Спорт': ['bench pressing', 'deadlifting', 'throwing discus', 'playing badminton', 'capoeira', 'hurling (sport)', 'wrestling', 'gymnastics tumbling', 'ice skating', 'jogging', 'bobsledding', 'golf putting', 'dribbling basketball', 'hockey stop', 'situp', 'playing basketball', 'high kick', 'playing chess', 'snowboarding', 'push up', 'throwing ball', 'archery', 'snatch weight lifting', 'playing ice hockey', 'swimming butterfly stroke', 'pull ups', 'punching bag', 'skiing crosscountry', 'punching person (boxing)', 'playing tennis', 'dunking basketball', 'arm wrestling', 'kicking soccer ball', 'shooting basketball', 'side kick', 'playing volleyball'],
    # Стиль и красота
    'Стиль и красота': ['cutting nails', 'making jewelry', 'trimming or shaving beard', 'brushing hair', 'waxing legs', 'curling hair', 'shaving legs', 'getting a haircut', 'braiding hair', 'dying hair', 'getting a tattoo', 'fixing hair', 'filling eyebrows', 'waxing back', 'waxing chest', 'doing nails', 'shaving head', 'waxing eyebrows', 'applying cream'],
    # Информационные технологии
    'Информационные технологии': ['texting', 'using computer', 'using remote controller (not gaming)', 'robot dancing'],
    # Телевидение
    'Телевидение': ['presenting weather forecast', 'news anchoring', 'giving or receiving award', 'answering questions'],
    # Путешествия
    'Путешествия': ['marching', 'cooking on campfire', 'abseiling', 'rock climbing', 'canoeing or kayaking', 'skydiving', 'ice fishing', 'riding mountain bike', 'riding camel', 'snorkeling', 'biking through snow', 'catching fish', 'riding elephant', 'riding a bike'],
    # Игры
    'Игры': ['assembling computer', 'using computer', 'playing keyboard', 'playing controller', 'playing cards'],
}

class PackPathway(torch.nn.Module):
    """
    Преобразование для конвертации кадров видео в список тензоров для модели SlowFast.
    """

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Выполнение временной выборки для медленного канала.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        return [slow_pathway, fast_pathway]


transform = ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x / 255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=side_size),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

# Продолжительность клипа
clip_duration = (num_frames * sampling_rate) / frames_per_second

In [6]:
def process_video(video_path):
    """
    Обрабатывает видео и возвращает список категорий tag_level1.
    """
    try:
        # Инициализация EncodedVideo и загрузка видео
        video = EncodedVideo.from_path(video_path)
        total_duration = video.duration

        if total_duration is None:
            print("Не удалось загрузить видео.")
            return None

        all_preds = []
        start_sec = 0

        # Обработка видео по клипам
        while start_sec < total_duration:
            end_sec = start_sec + clip_duration
            if end_sec > total_duration:
                end_sec = total_duration

            # Загрузка клипа
            video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

            # Проверяем, что клип содержит данные
            if video_data is None or video_data["video"] is None:
                break

            # Применение преобразований
            video_data = transform(video_data)

            # Подготовка входных данных для модели
            inputs = video_data["video"]
            inputs = [i.to(device)[None, ...] for i in inputs]

            # Получение предсказаний от модели
            with torch.no_grad():
                preds = model(inputs)

            # Добавление предсказаний в список
            all_preds.append(preds.cpu())

            # Переход к следующему клипу
            start_sec += clip_duration

        if not all_preds:
            print("Не удалось получить предсказания.")
            return None

        # Агрегация предсказаний по всему видео
        final_preds = torch.cat(all_preds).mean(dim=0, keepdim=True)

        # Применение Softmax для получения вероятностей
        post_act = torch.nn.Softmax(dim=1)
        final_preds = post_act(final_preds)

        # Получение топ-5 предсказанных классов
        pred_classes = final_preds.topk(k=5).indices[0]

        # Отображение предсказанных классов на названия
        pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes]

        # Преобразование названий действий в нижний регистр для корректного сравнения
        pred_class_names_lower = [name.lower() for name in pred_class_names]

        # Инициализация списка категорий
        tag_level1 = []

        # Проверка предсказаний на соответствие категориям
        for category, actions in categories.items():
            # Преобразуем действия в нижний регистр для сравнения
            actions_lower = [action.lower() for action in actions]
            if any(pred in actions_lower for pred in pred_class_names_lower):
                tag_level1.append(category)

        # Если ни одна категория не совпала, добавляем 'другое'
        if not tag_level1:
            tag_level1.append('другое')

        return tag_level1

    except Exception as e:
        print(f"Ошибка при обработке видео: {e}")
        return None

In [7]:
# Пример использования функции
video_path = 'D:/RUTUBE_HACK/video/4a6d6e98413508867dc1adafef8c5e46.mp4'
categories_detected = process_video(video_path)
print("Обнаруженные категории:", categories_detected)

Обнаруженные категории: ['Музыка и аудио']
